In [1]:
!pip install instaloader pandas joblib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 2.4 MB/s eta 0:00:00


In [2]:
import instaloader
import pandas as pd
import re
import joblib
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
model = joblib.load('/content/cyberbully_model.pkl')
vectorizer = joblib.load('/content/tfidf_vectorizer.pkl')

def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9_]+','',text)
    text = re.sub(r'#','',text)
    text = re.sub(r'RT[\s]+','',text)
    text = re.sub(r'https?:\/\/\S+','',text)
    text = re.sub(r'[^A-Za-z\s]','',text)
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

def classify_comment(text):
    cleaned = clean_text(text)
    vector = vectorizer.transform([cleaned])
    pred = model.predict(vector)[0]
    return {0:"🚫 Hate Speech", 1:"⚠️ Offensive", 2:"✅ Neutral"}[pred]


In [10]:
import instaloader
L = instaloader.Instaloader()

USERNAME = "681kihtrak"
PASSWORD = "Palinakarthik@18"

# Login once; it will store a session file
L.login(USERNAME, PASSWORD)

# Optional: save session to reuse
L.save_session_to_file()

shortcode = "C3Xk5W5s8sH"
post = instaloader.Post.from_shortcode(L.context, shortcode)

comments = []
for c in post.get_comments():
    comments.append({'user': c.owner.username, 'comment': c.text})

import pandas as pd
pd.DataFrame(comments).head()


LoginException: Login error: "fail" status, message "CSRF token missing or incorrect".

In [ ]:
comments_df['Prediction'] = comments_df['comment'].apply(classify_comment)
comments_df.head(10)
